In [ ]:
######################   Importar librerías: #######################
"""-  'pandas' para la manipulación y análisis de datos.
- 'random'  para generar números y secuencias aleatorias.
- La clase 'datetime' para trabajar con fechas y horas.
- 'os' para interactuar con el sistema operativo.
- 'logging' para generar mensajes de log.
- 'time' para funciones relacionadas con el tiempo."""

import pandas as pd
import random
from datetime import datetime
import os
import logging
import time
import platform
import getpass

# Configurar el logger
user = getpass.getuser()
system = platform.system()
platform_details = platform.platform()

log_dir = '.'  # Cambia este directorio si quieres guardar el log en otra ubicación
log_filename = os.path.join(log_dir, 'event_log.log')

logging.basicConfig(
    filename=log_filename,
    level=logging.INFO,
    format='%(asctime)s\t%(message)s',
    datefmt='%Y-%m-%d %H:%M:%S.%f'
)

# Escribir el encabezado del log
logging.info(f'Usuario: {user}')
logging.info(f'Sistema Operativo: {system}')
logging.info(f'Plataforma: {platform_details}')
logging.info('Instrucción\tTiempo (segundos)')

# Función para registrar eventos
def log_event(event, start_time):
    end_time = time.time()
    duration = end_time - start_time
    logging.info(f'{event}\t{duration:.6f}')

acciones = 0

######################## Carga de Archivos: #########################

""" Se cargan los nombres y apellidos desde archivos CSV en GitHub.
La función read_csv permite leer los datos desde las direcciones url.
Extraemos la columna 'Nombre' y 'Apellidos' y se convierten en listas"""

start_time = time.time()
nombres = (pd.read_csv('https://github.com/mivem123/Trabajo_final/raw/64b01cf742950b2bd075fbdffd840028f05a809d/nombres.csv')['Nombre']).tolist()
log_event('Cargar nombres', start_time)

start_time = time.time()
apellidos = (pd.read_csv('https://github.com/mivem123/Trabajo_final/raw/64b01cf742950b2bd075fbdffd840028f05a809d/apellidos.csv')['Apellidos']).tolist()
log_event('Cargar apellidos', start_time)

#################### Generar combinaciones únicas de nombres y apellidos ##################

""" Se crea un conjunto vacío 'estudiantes' para almacenar las combinaciones.
Con el bucle 'while' podremos generar las combinaciones hasta que se tengan 1000 elementos.
La función 'random.choice' nos permite seleccionar aleatoriamente nombres y apellidos de las listas.
"""

estudiantes = set()
start_time = time.time()
while len(estudiantes) < 1000:
    nombre = random.choice(nombres)
    nombre2 = random.choice(nombres)
    apellido = random.choice(apellidos)
    apellido2 = random.choice(apellidos)
    estudiantes.add(f"{nombre} {nombre2} {apellido} {apellido2}")
log_event('Generar combinaciones únicas de nombres y apellidos', start_time)

#################### Asignar semestres proporcionalmente #######################

""" Se define una lista 'semestres' vacía para almacenar los semestres de cada estudiante.
Con el ciclo 'for' se extiende la lista semestres con el numero de estudiantes correspondientes segun las proporciones.
Cada proporción se multiplica por 10 para obtener el numero entero de estudiantes por semestre.
"""
semestres = []
proporciones = [14, 13, 12, 11, 10, 10, 9, 8, 7, 6]  # Ejemplo de proporciones
start_time = time.time()
for i, proporcion in enumerate(proporciones):
    semestres.extend([i+1] * (proporcion * 10))
log_event('Asignar semestres proporcionalmente', start_time)

################# Generar datos ficticios y guardar en CSV ###########

"""Se crea la lista vacía 'data' para almacenar los datos de los estudiantes.
con el 'for' se recorre el conjunto de estudiantes y la lista de semestres simultaneamente.
Para cada estudiante se crea un diccionario: nombre completo, semestre y correo.
El correo tiene la estrctura de combinar los dos primeros nombres en minuscula y el dominio de la UdeA."""

data = []
start_time = time.time()
for estudiante, semestre in zip(estudiantes, semestres):
    data.append({
        'Nombre': estudiante.split()[0] + " " + estudiante.split()[1] + " " + estudiante.split()[2] + " " + estudiante.split()[3],
        'Semestre': semestre,
        'Cédula': ''.join([str(random.randint(0, 9)) for _ in range(10)]),
        'Correo': f"{estudiante.split()[0].lower()}.{estudiante.split()[1].lower()}@udea.edu.co"
    })
log_event('Generar datos ficticios', start_time)

################ Convertir la lista en un DataFrame de pandas y guardarlo en un CSV #######

"""Se utiliza la funcion DataFrame de pandas para crearlo desde la lista 'data'.
Se guarda el DataFrame en un Archivo CSV utilizando la funcion 'to_csv' sin índice."""

start_time = time.time()
df_estudiantes = pd.DataFrame(data)
df_estudiantes.to_csv('estudiantes.csv', index=False)
log_event('Guardar estudiantes en CSV', start_time)
acciones += 1

############ Definición de los datos de porcentajes y cupos por semestre ############

"""Se crea un diccionario 'datos_semestre' donde: La clave es numero de semestre.
Y el valor es un diccionario con el porcentaje y el cupo máximo de estudiantes por aula para ese semestre"""

datos_semestres = {
    1: {'porcentaje': 0.14, 'cupo': 30},
    2: {'porcentaje': 0.13, 'cupo': 30},
    3: {'porcentaje': 0.12, 'cupo': 30},
    4: {'porcentaje': 0.11, 'cupo': 25},
    5: {'porcentaje': 0.10, 'cupo': 25},
    6: {'porcentaje': 0.10, 'cupo': 25},
    7: {'porcentaje': 0.09, 'cupo': 20},
    8: {'porcentaje': 0.08, 'cupo': 20},
    9: {'porcentaje': 0.07, 'cupo': 20},
    10: {'porcentaje': 0.06, 'cupo': 10}
}

############ Leer el archivo de Excel con la malla curricular desde GitHub ##############

"""Utilizamos la funcion read_excel para leer los datos desde GitHub.
Se convierte el DataFrame resultante en una lista de diccionarios con la función 'to_dict'
y el parametro 'orient = 'records''"""

start_time = time.time()
datos = pd.read_excel('https://github.com/mivem123/Trabajo_final/raw/64b01cf742950b2bd075fbdffd840028f05a809d/Asignaturas.xlsx')
asignaturas = datos.to_dict(orient='records')
log_event('Leer malla curricular', start_time)

################ Función para generar el código de la asignatura #################

"""La funcion 'generar_codigo_asignatura' toma como parametros: nombre de la asignatura, el semestre, los creditos y un num consecutivo.
Genera un codigo para la asignatura basado en: tomar las primeras letras de la palabra del nombre (max 3), el semestre, los creditos y el consecutivo """

def generar_codigo_asignatura(nombre, semestre, creditos, consecutivo):
    nombre_corto = ''.join([palabra[0].upper() for palabra in nombre.split()][:3])
    return f"{nombre_corto}{semestre}{creditos}{consecutivo:d}"

################ Crear una lista para almacenar la planificación #########

planificacion = []

####################### Crear la estructura de carpetas ########################

"""Se define la ruta 'base_dir' donde se guardarán los archivos.
Si la carpeta no existe, se crea utilizando 'os.makedirs'. """

base_dir = 'Trabajo_Final'
start_time = time.time()
if not os.path.exists(base_dir):
    os.makedirs(base_dir)
log_event('Crear estructura de carpetas base', start_time)

"""Se recorre cada asignatura en la malla curricular. El 'for' itera sobre cada asignatura en la lista.
Y para cada una se obtiene el nombre, semestre y los creditos """

for asignatura in asignaturas:
    nombre = asignatura['Asignatura']
    semestre = asignatura['Nivel']
    creditos = asignatura['Créditos']

    ######################### Filtrar estudiantes por semestre ################

    """Se crea la lista 'estudiantes_semestre' que contiene los estudiantes cuyo semestre coincide con el actual.
    El limite de estudiantes se obtiene del diccionario 'datos_semestres'
    Se calcula el número de grupos necesarios dividiendo el número total de estudiantes entre el limite del aula. """

    estudiantes_semestre = [est for est in data if est['Semestre'] == semestre]
    limite_por_aula = datos_semestres[semestre]['cupo']
    num_grupos = -(-len(estudiantes_semestre) // limite_por_aula)

    """Se utiliza el 'for' para iterar sobre el numero de grupos necesarios.
    Para cada grupo se genera un codigo de asignatura utilizando la funcion 'generar_codigo_asignatura'
    Se definen las HTD y las HTI segun los creditos de la asignatura.
    Se calcula el num total de estudiantes en el grupo actual. """

    for i in range(num_grupos):
        start_time = time.time()
        codigo_asignatura = generar_codigo_asignatura(nombre, semestre, creditos, i+1)
        htd = 96 if creditos == 4 else 64 if creditos == 3 else 32 if creditos == 2 else 16 if creditos ==1 else 0
        hti = 120 if creditos == 4 else 80 if creditos == 3 else 64 if creditos == 2 else 32 if creditos ==1 else 0
        nte = min(limite_por_aula, len(estudiantes_semestre) - i * limite_por_aula)

        #### Añadir los datos de la planificación del grupo a la lista 'planificacion' #####3

        """Se crea un diccionario con la info del grupo y se añade a la lista.
        La info incluye: codigo de la materia, HTD y HTI, el
        num total de estudiantes en el grupo y en el semestre, el codigo del curso CC,
        numero total de cursos asignados TCA, y la fecha de creación FC."""

        planificacion.append({
            'Código Asignatura': codigo_asignatura,
            'Horas de trabajo docente (HTD)': htd,
            'Horas de trabajo independiente (HTI)': hti,
            'Número total de estudiantes por grupo': nte,
            'Número total de estudiantes (NTE)':len(estudiantes_semestre),
            'Código del curso (CC)': i + 1,
            'Total de cursos asignados (TCA)': num_grupos,
            'Fecha de creación (FC)': datetime.now().strftime('%Y-%m-%d')
        })
        log_event(f'Planificar grupo {codigo_asignatura}', start_time)
        acciones += 1

        ######################## Seleccionar estudiantes para el grupo actual #########################

        """Se seleccionan los estudiantes correspondientes al grupo actual utilizando
        el indice del bucle."""

        start_time = time.time()
        estudiantes_grupo = estudiantes_semestre[i * limite_por_aula: (i + 1) * limite_por_aula]

        ##############3 Crear carpeta para el semestre ####################

        """Se define la ruta 'semestre_dir' para el semestre """

        semestre_dir = os.path.join(base_dir, f'Semestre_{semestre}')
        if not os.path.exists(semestre_dir):
            os.makedirs(semestre_dir)

        # Crear carpeta para la asignatura
        asignatura_dir = os.path.join(semestre_dir, nombre.replace(' ', '_'))
        if not os.path.exists(asignatura_dir):
            os.makedirs(asignatura_dir)

        # Crear DataFrame para el grupo
        df_grupo = pd.DataFrame(estudiantes_grupo)

        # Generar nombre de archivo
        nombre_archivo = f"{codigo_asignatura}-{nombre.replace(' ', '')}-{len(estudiantes_grupo)}-{i + 1}"

        # Guardar CSV
        df_grupo.to_csv(os.path.join(asignatura_dir, f"{nombre_archivo}.csv"), index=False)

        # Guardar Excel
        df_grupo.to_excel(os.path.join(asignatura_dir, f"{nombre_archivo}.xlsx"), index=False)

        log_event(f'Guardar archivos del grupo {codigo_asignatura}', start_time)
        acciones += 2

# Crear un DataFrame y guardar el archivo CSV de la planificación
start_time = time.time()
df_planificacion = pd.DataFrame(planificacion)
df_planificacion.to_csv('planificacion.csv', index=False)
log_event('Guardar planificación en CSV', start_time)
acciones += 1

# Registrar la cantidad de procedimientos realizados y el tipo de acción
logging.info(f'Total de acciones: {acciones}')
logging.info(f'Tipos de acciones: contar archivos, mover, renombrar, guardar')
